# Dummy Tweet Scraping

### Import Libraries

In [1]:
from tweepy import Stream
from tweepy import OAuthHandler
#from tweepy.streaming import StreamListener
import tweepy
import json
import pandas as pd
import csv
import string
import time
import datetime
import numpy as np
from cmath import nan

### Account Authentification

In [2]:
# Authenticating Twitter API
# Obtain your Twitter credentials from your twitter developer account

bearer_token = 'AAAAAAAAAAAAAAAAAAAAAEGchQEAAAAAZSYFv1nyLDV81YAKEfDr1fVrlho%3DWKBvyLhQ4CeHrlBRtecAetYkB1ZnAjI3Zydb1516fkIzKhS4vh'

client = tweepy.Client(bearer_token, wait_on_rate_limit=True)

### Search Criteria
This is just an arbitary search criteria for Dummy results with the word protest in them

In [3]:
#search_words = '(protest OR strike OR march OR rally OR #feesmustfall OR #zumamustfall) -is:retweet lang:en place_country:ZA'
#search_words = 'protest -is:retweet lang:en place_country:ZA'
search_words = '-is:retweet lang:en place_country:ZA'
date_since = datetime.datetime(2022,9,23)
date_before = datetime.datetime(2022,9,30)
numTweets = 500

Scrape Tweets

In [4]:
# Scrape Tweets
# Note for the final versions we should use -> until_id -> allos to resume scraping from a particualar tweet ID
protest_tweets = []
count = 0 
for response in tweepy.Paginator(client.search_all_tweets, 
                                 query = search_words,
                                 user_fields = ['public_metrics'],
                                 tweet_fields = ['created_at', 'geo', 'public_metrics', 'text'],
                                 place_fields = ['id', 'geo', 'name', 'country_code', 'place_type', 'full_name', 'country'],
                                 expansions = ['author_id', 'geo.place_id'],
                                 start_time = date_since,
                                 end_time = date_before,
                                 max_results=numTweets
                              ):
   time.sleep(1)
   protest_tweets.append(response)

Populate Dataframe with tweet data

In [5]:
error_count = 0
result = []
user_dict = {}
place_dict = {}
# Loop through each response object
for response in protest_tweets:
    # Take all of the users, and put them into a dictionary of dictionaries with the info we want to keep
    for user in response.includes['users']:
        user_dict[user.id] = {'followers': user.public_metrics['followers_count'],
                              'tweets': user.public_metrics['tweet_count'],
                             }
    
    place_dict = {p['id']: p for p in response.includes['places']}

    for tweet in response.data:
        # For each tweet, find the author's information
        author_info = user_dict[tweet.author_id]
        # Put all of the information we want to keep in a single dictionary for each tweet

        try:
            if place_dict[tweet.geo['place_id']]:
                place = place_dict[tweet.geo['place_id']]   
            else:
                place = nan

            result.append({'tweet_id': tweet.id,
                       'author_followers': author_info['followers'],
                       'text': tweet.text,
                       'created_at': tweet.created_at,
                       'retweets': tweet.public_metrics['retweet_count'],
                       'replies': tweet.public_metrics['reply_count'],
                       'likes': tweet.public_metrics['like_count'],
                       'quote_count': tweet.public_metrics['quote_count'],
                       'place_id': tweet.geo,
                       'place_name': place.full_name,
                       'bbox': place.geo['bbox']

                      })

        except:
            error_count = error_count + 1


print(error_count, " result(s) were excluded due to invalid data points.")        

# Change this list of dictionaries into a dataframe             
df = pd.DataFrame(result)                

0  result(s) were excluded due to invalid data points.


In [6]:
# (number of tweets, number of attributes)
df.shape

(128910, 12)

Save data to csv

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 128910 entries, 0 to 128909
Data columns (total 12 columns):
 #   Column            Non-Null Count   Dtype              
---  ------            --------------   -----              
 0   tweet_id          128910 non-null  int64              
 1   author_id         128910 non-null  int64              
 2   author_followers  128910 non-null  int64              
 3   text              128910 non-null  object             
 4   created_at        128910 non-null  datetime64[ns, UTC]
 5   retweets          128910 non-null  int64              
 6   replies           128910 non-null  int64              
 7   likes             128910 non-null  int64              
 8   quote_count       128910 non-null  int64              
 9   place_id          128910 non-null  object             
 10  place_name        128910 non-null  object             
 11  bbox              128910 non-null  object             
dtypes: datetime64[ns, UTC](1), int64(7), object(

In [8]:
df.head(2)

,tweet_id,author_id,author_followers,text,created_at,retweets,replies,likes,quote_count,place_id,place_name,bbox
0,1575636537294737408,1005713558955610112,3276,@NduLindani Thank you I’ll go to the doctor ag...,2022-09-29 23:59:59+00:00,0,1,0,0,{'place_id': '9d7cae88ff6a29f5'},"Durban, South Africa","[30.8789097, -30.055938, 31.0662492, -29.7453336]"
1,1575636504185274369,343624915,138,@RiebvJanbeeck @marcel__za Always good,2022-09-29 23:59:51+00:00,0,0,1,0,{'place_id': '46c1b1ab24d7e11a'},"Midrand, South Africa","[28.0140761, -26.0572574, 28.1930256, -25.9195..."


In [9]:
df.to_csv('CSV/Scrape_Tweets.csv')